In [2]:
# Add script to include project root in sys.path

import sys
from pathlib import Path

# Add project root (one level up from notebooks/)
project_root = Path(__file__).resolve().parent.parent if "__file__" in globals() else Path.cwd().parent
sys.path.append(str(project_root))

In [3]:
# Import necessary libraries

from src.gradcam import GradCAM
from torchvision import models, transforms
from PIL import Image
import torch
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
# Load pretrained model

model = models.resnet50(weights="IMAGENET1K_V1")
gradcam = GradCAM(model, target_layer_name="layer4")